In [62]:
%load_ext autoreload
%autoreload 2

In [1]:
import json

In [2]:
from collabmem.constants import REPO_ROOT

In [11]:
COMPOSED_EVAL_OUT_FILE = REPO_ROOT / "outputs/composed_gsm8k_eval_artifacts.json"

In [12]:
with open(COMPOSED_EVAL_OUT_FILE, "r") as f:
    composed_eval_data = json.load(f)

In [3]:
with open(REPO_ROOT / "outputs/back_half_code_repr.json") as f:
    back_half_code_repr = json.load(f)

In [5]:
back_half_code_repr.keys()

dict_keys(['problems', 'code_representations'])

In [ ]:
back_half_code_repr["code_representations"]

[{'example_id': 283,
  'question': 'Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?',
  'answer_value': 11.0,
  'code': 'def solve():\n    n = 5\n    first_price = 1\n    increment = 1\n    total = n * (2 * first_price + (n - 1) * increment) / 2\n    profit = 4\n    prize = total - profit\n    return float(prize)',
  'raw_model_output': 'def solve():\n    n = 5\n    first_price = 1\n    increment = 1\n    total = n * (2 * first_price + (n - 1) * increment) / 2\n    profit = 4\n    prize = total - profit\n    return float(prize)',
  'execution_output': {'stdout': '11.0\n', 'stderr': '', 'returncode': 0},
  'correct': True},
 {'example_id': 3402,
  'question': 'A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, an

In [24]:
count = 0
rounding_free = []
for item in back_half_code_repr["code_representations"]:
    code = item["code"]
    # search for "//" double slash as integer division indicator in python
    if "//" in code:
        print(code)
        count += 1
    else:
        rounding_free.append(item["example_id"])
print(f"Found {count} instances of '//' in code representations.")

def solve():
    prep_time_per_dish_min = 20
    time_available_hours = 2
    people_per_dish = 5

    time_available_min = time_available_hours * 60
    num_dishes = time_available_min // prep_time_per_dish_min
    answer = float(num_dishes * people_per_dish)
    return answer
def solve():
    price_per_peach = 0.40
    num_peaches = 400
    discount_per_10 = 2.0
    threshold = 10.0

    total = num_peaches * price_per_peach
    discount = (total // threshold) * discount_per_10
    answer = total - discount
    return answer
Found 2 instances of '//' in code representations.


In [25]:
rounding_free_q = [
    ex
    for ex in back_half_code_repr["code_representations"]
    if ex["example_id"] in rounding_free
]

In [34]:
print(rounding_free_q[0]["question"])
print(rounding_free_q[0]["code"])
print(rounding_free_q[0]["answer_value"])

Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?
def solve():
    n = 5
    first_price = 1
    increment = 1
    total = n * (2 * first_price + (n - 1) * increment) / 2
    profit = 4
    prize = total - profit
    return float(prize)
11.0


In [33]:
rounding_free[0]

283

In [13]:
composed_eval_data["examples"][3]

{'composed_question': 'Let X be the answer to Q1:\nQ1: John wants to start a zoo. He has 15 snakes. He has twice as many monkeys as he does snakes. He has 5 fewer lions than he does monkeys. John has 8 more pandas than he does lions. John has 1/3 as many dogs as he does pandas. How many total animals does John have?\nSolve it and use the value of X to solve Q2. Explain your answer step by step\nQ2: The pet store can buy a goldfish for $0.25 and sell it for $0.75. The owner plans to use the profits from goldfish sales to buy a new tank, which costs X dollars. After one week, he is 45% short of the price. How many goldfish did he sell that week?',
 'q1_id': 4833,
 'q2_id': 6656,
 'q1_answer': 114.0,
 'composed_answer': 125.4,
 'model_response': 'Step 1 — Q1: number of each animal\n- Snakes = 15\n- Monkeys = 2 × 15 = 30\n- Lions = 30 − 5 = 25\n- Pandas = 25 + 8 = 33\n- Dogs = (1/3) × 33 = 11\nTotal animals X = 15 + 30 + 25 + 33 + 11 = 114\n\nSo X = 114.\n\nStep 2 — Q2\n- Profit per goldfi

In [14]:
dummies = []
for idx, ex in enumerate(composed_eval_data["examples"]):
    if ex["q1_composed_correct"] and not ex["q2_composed_correct"]:
        dummies.append((idx, ex))

In [15]:
dummies

[(3,
  {'composed_question': 'Let X be the answer to Q1:\nQ1: John wants to start a zoo. He has 15 snakes. He has twice as many monkeys as he does snakes. He has 5 fewer lions than he does monkeys. John has 8 more pandas than he does lions. John has 1/3 as many dogs as he does pandas. How many total animals does John have?\nSolve it and use the value of X to solve Q2. Explain your answer step by step\nQ2: The pet store can buy a goldfish for $0.25 and sell it for $0.75. The owner plans to use the profits from goldfish sales to buy a new tank, which costs X dollars. After one week, he is 45% short of the price. How many goldfish did he sell that week?',
   'q1_id': 4833,
   'q2_id': 6656,
   'q1_answer': 114.0,
   'composed_answer': 125.4,
   'model_response': 'Step 1 — Q1: number of each animal\n- Snakes = 15\n- Monkeys = 2 × 15 = 30\n- Lions = 30 − 5 = 25\n- Pandas = 25 + 8 = 33\n- Dogs = (1/3) × 33 = 11\nTotal animals X = 15 + 30 + 25 + 33 + 11 = 114\n\nSo X = 114.\n\nStep 2 — Q2\n- 

In [16]:
dummy = dummies[0][1]
print(dummy["composed_question"])
print(f"\n---\n\nexpected answer: {dummy['composed_answer']}\n---\n")
print(f"model output:\n{dummy['model_response']}")

Let X be the answer to Q1:
Q1: John wants to start a zoo. He has 15 snakes. He has twice as many monkeys as he does snakes. He has 5 fewer lions than he does monkeys. John has 8 more pandas than he does lions. John has 1/3 as many dogs as he does pandas. How many total animals does John have?
Solve it and use the value of X to solve Q2. Explain your answer step by step
Q2: The pet store can buy a goldfish for $0.25 and sell it for $0.75. The owner plans to use the profits from goldfish sales to buy a new tank, which costs X dollars. After one week, he is 45% short of the price. How many goldfish did he sell that week?

---

expected answer: 125.4
---

model output:
Step 1 — Q1: number of each animal
- Snakes = 15
- Monkeys = 2 × 15 = 30
- Lions = 30 − 5 = 25
- Pandas = 25 + 8 = 33
- Dogs = (1/3) × 33 = 11
Total animals X = 15 + 30 + 25 + 33 + 11 = 114

So X = 114.

Step 2 — Q2
- Profit per goldfish = $0.75 − $0.25 = $0.50
- He is 45% short, so he has 55% of the tank price: money raised

In [ ]:
cr_dict = {}
for ex in back_half_code_repr["code_representations"]:
    cr_dict[ex["example_id"]] = ex

In [23]:
print(cr_dict[dummy["q2_id"]]["code"])

def solve():
    cost_price = 0.25
    sell_price = 0.75
    tank_cost = 100.0
    short_percent = 0.45
    amount_collected = (1 - short_percent) * tank_cost
    profit_per_fish = sell_price - cost_price
    number_sold = amount_collected / profit_per_fish
    return number_sold


In [59]:
cases_where_both_incorrect = []
for idx, ex in enumerate(composed_eval_data["examples"]):
    if not ex["q1_composed_correct"] and not ex["q2_composed_correct"]:
        cases_where_both_incorrect.append((idx, ex))
cases_where_both_incorrect

[(9,
  {'composed_question': 'Let X be the answer to Q1:\nQ1: New York recorded 5000 new coronavirus cases on a particular week. In the second week, half as many new coronaviruses cases as the first week was recorded by the state. In the third week, 2000 more cases were recorded in the state. What is the total number of recorded new coronaviruses in the state after the three weeks?\nSolve it and use the value of X to solve Q2. Explain your answer step by step\nQ2: Mabel is counting sharks in the ocean. She knows that of the fish that she sees, 25% will be sharks and 75% will be another type of fish. On day one she counts X fish. On day 2 she counts three times as many. How many sharks did she count over those two days?',
   'q1_id': 3341,
   'q2_id': 3788,
   'q1_answer': 9500.0,
   'composed_answer': 9500.0,
   'model_response': 'Q1 step-by-step:\n- Week 1: 5000 cases.\n- Week 2: half as many as week 1 = 5000/2 = 2500 cases.\n- Week 3: 2000 more than week 2 = 2500 + 2000 = 4500 cases.

In [35]:
from collabmem.nhop_compose import compose_multi_hop_problems

In [ ]:
from collabmem.compose_q import CodeRepresentation, QAExample

In [ ]:
# def compose_multi_hop_problems(
#     chains: List[Tuple[QAExample, List[Tuple[QAExample, CodeRepresentation]]]],
#     client: LLMClient,
#     gen_cfg: GenerationConfig,
#     model: str = "gpt-5-mini-2025-08-07",
# ) -> List[Optional[MultiHopComposedExample]]:

In [39]:
INIT_OUT_FILE = REPO_ROOT / "outputs/gsm8k_baseline_predictions.json"
# load selected problems file
with open(INIT_OUT_FILE, "r") as f:
    data = json.load(f)

# load problems
problems = [QAExample(**prob) for prob in data["problems"]]

In [ ]:
problem_dict = {prob.id: prob for prob in problems}

In [ ]:
rf_dict = {idx: problem_dict[idx] for idx in rounding_free}
rf_probs = list(rf_dict.values())

In [48]:
import random

In [ ]:
import importlib

import collabmem.nhop_compose

importlib.reload(collabmem.nhop_compose)
from collabmem.nhop import MultiHopComposedExample

In [ ]:
random.seed(42)
CHAIN_LEN = 3
random.sample(rf_probs, CHAIN_LEN)
chains = [
    (
        rf_probs[0],
        [(rf_probs[i], cr_dict[rf_probs[i].id]) for i in range(1, CHAIN_LEN)],
    ),
]

In [ ]:
res = MultiHopComposedExample(
    ids=[283, 3402, 1376],
    questions=[
        "Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?",
        "A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?",
        "Bronson decides to collect Oak Leaves from around his Neighborhood. He collects 12 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?",
    ],
    answer_values_original=[11.0, 2.0, 15.0],
    composed_question="Let X be the answer to Q1:\nQ1: \nLet X be the answer to Q1:\nQ1: \nLily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?\n\nSolve Q1 and use the value of X to solve Q2. Explain your answer step by step.\nQ2: \nA movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $11. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?\n\nSolve it and use the value of X to solve Q2. Explain your answer step by step.\nQ2: \nBronson decides to collect Oak Leaves from around his Neighborhood. He collects 11 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?",
    composed_answer_value=14.400000000000002,
    composed_code="def solve_revised():\n    thursday = 11.0\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow",
)

In [ ]:
print(res.composed_question)

Let X be the answer to Q1:
Q1: 
Let X be the answer to Q1:
Q1: 
Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?

Solve Q1 and use the value of X to solve Q2. Explain your answer step by step.
Q2: 
A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $11. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?

Solve it and use the value of X to solve Q2. Explain your answer step by step.
Q2: 
Bronson decides to collect Oak Leaves from around his Neighborhood. He collects 11 on Thursd

In [ ]:
# v2 = MultiHopComposedExample(ids=[283, 3402, 1376], questions=['Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?', 'A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?', 'Bronson decides to collect Oak Leaves from around his Neighborhood. He collects 12 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?'], answer_values_original=[11.0, 2.0, 15.0], composed_question='Let X_i be the answer to Q_i:\nSolve Q_i and use the value of X_i to solve Q_{i+1}. Explain your answer step by step.\nQ1: Lily has 5 lottery tickets to sell. She sells the first ticket for $1. She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?\nQ2: Bronson decides to collect Oak Leaves from around his Neighborhood. He collects X1 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?\nQ3: If the total yellow leaves from Q2 equals X2 and he distributes them equally among 5 friends, how many leaves does each friend receive? Explain your answer step by step.', composed_answer_value=15.300000000000002, composed_code='def solve_revised():\n    thursday = 12.5\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow')
v3 = MultiHopComposedExample(
    ids=[283, 3402, 1376],
    questions=[
        "Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?",
        "A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?",
        "Bronson decides to collect Oak Leaves from around his Neighborhood. He collects 12 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?",
    ],
    answer_values_original=[11.0, 2.0, 15.0],
    composed_question="Let X_i be the answer to Q_i:\nSolve Q_i and use the value of X_i to solve Q_{i+1}. Explain your answer step by step.\nQ1: Lily has 5 lottery tickets to sell. She sells the first ticket for $1. She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?\nQ2: Bronson decides to collect Oak Leaves from around his Neighborhood. He collects X1 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?\nQ3: If the total yellow leaves from Q2 equals X2 and he distributes them equally among 5 friends, how many leaves does each friend receive? Explain your answer step by step.",
    composed_answer_value=15.300000000000002,
    composed_code="def solve_revised():\n    thursday = 12.5\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow",
    intermediate_answers=[12.5, 15.300000000000002],
)

In [71]:
v3.__dict__.keys()

dict_keys(['ids', 'questions', 'answer_values_original', 'composed_question', 'composed_answer_value', 'composed_code', 'intermediate_answers'])

In [73]:
print(v3.answer_values_original)

[11.0, 2.0, 15.0]


In [72]:
print(v3.intermediate_answers)

[12.5, 15.300000000000002]


In [75]:
print(v3.composed_code)

def solve_revised():
    thursday = 12.5
    friday = 13
    total = thursday + friday
    percent_brown = 0.2
    percent_green = 0.2
    percent_yellow = 1 - percent_brown - percent_green
    yellow = total * percent_yellow
    return yellow


In [67]:
print(v3.composed_question)

Let X_i be the answer to Q_i:
Solve Q_i and use the value of X_i to solve Q_{i+1}. Explain your answer step by step.
Q1: Lily has 5 lottery tickets to sell. She sells the first ticket for $1. She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?
Q2: Bronson decides to collect Oak Leaves from around his Neighborhood. He collects X1 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?
Q3: If the total yellow leaves from Q2 equals X2 and he distributes them equally among 5 friends, how many leaves does each friend receive? Explain your answer step by step.


In [ ]:
# new_res = [MultiHopComposedExample(ids=[283, 3402, 1376], questions=['Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?', 'A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?', 'Bronson decides to collect Oak Leaves from around his Neighborhood. He collects 12 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?'], original_codes=[None, 'def solve():\n    ticket = 8.0\n    popcorn = ticket - 3.0\n    drink = popcorn + 1.0\n    candy = drink / 2.0\n    normal_total = ticket + popcorn + drink + candy\n    deal_price = 20.0\n    answer = normal_total - deal_price\n    return answer', 'def solve():\n    thursday = 12\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow'], answer_values_original=[11.0, 2.0, 15.0], composed_question='Let X_i be the answer to Q_i:\nSolve Q_i and use the value of X_i to solve Q_{i+1}. Explain your answer step by step.\nQ1: Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?\nQ2: A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs X_1 dollars, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?\nQ3: Bronson decides to collect Oak Leaves from around his Neighborhood. He collects X_1 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?', composed_answer_value=15.300000000000002, composed_code='def solve_revised():\n    thursday = 12.5\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow', intermediate_answers=[11.0, 12.5, 15.300000000000002], rewritten_subquestions=['A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs X_1 dollars, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?', 'Bronson decides to collect Oak Leaves from around his Neighborhood. He collects X_1 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?'], revised_codes=['def solve_revised():\n    ticket = 11.0\n    popcorn = ticket - 3.0\n    drink = popcorn + 1.0\n    candy = drink / 2.0\n    normal_total = ticket + popcorn + drink + candy\n    deal_price = 20.0\n    answer = normal_total - deal_price\n    return answer', 'def solve_revised():\n    thursday = 12.5\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow'])]
# new_res = [MultiHopComposedExample(ids=[283, 3402, 1376], questions=['Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?', 'A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?', 'Bronson decides to collect Oak Leaves from around his Neighborhood. He collects 12 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?'], original_codes=[None, 'def solve():\n    ticket = 8.0\n    popcorn = ticket - 3.0\n    drink = popcorn + 1.0\n    candy = drink / 2.0\n    normal_total = ticket + popcorn + drink + candy\n    deal_price = 20.0\n    answer = normal_total - deal_price\n    return answer', 'def solve():\n    thursday = 12\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow'], answer_values_original=[11.0, 2.0, 15.0], composed_question='Let X_i be the answer to Q_i:\nSolve Q_i and use the value of X_i to solve Q_{i+1}. Explain your answer step by step.\nQ1: Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?\nQ2: A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $X_1. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally? (Use the value X_1 = 11.0.)\nQ3: Bronson decides to collect Oak Leaves from around his Neighborhood. He collects 11 (the value of X_2) on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?', composed_answer_value=14.400000000000002, composed_code='def solve_revised():\n    thursday = 11.0\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow', intermediate_answers=[11.0, 11.0, 14.400000000000002], rewritten_subquestions=['A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $X_1. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally? (Use the value X_1 = 11.0.)', 'Bronson decides to collect Oak Leaves from around his Neighborhood. He collects 11 (the value of X_2) on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?'], revised_codes=['def solve_revised():\n    ticket = 8.0\n    popcorn = ticket - 3.0\n    drink = popcorn + 1.0\n    candy = drink / 2.0\n    normal_total = ticket + popcorn + drink + candy\n    deal_price = 11.0\n    answer = normal_total - deal_price\n    return answer', 'def solve_revised():\n    thursday = 11.0\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow'])]
new_res = [
    MultiHopComposedExample(
        ids=[283, 3402, 1376],
        questions=[
            "Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?",
            "A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs $8, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?",
            "Bronson decides to collect Oak Leaves from around his Neighborhood. He collects 12 on Thursday and 13 on Friday. 20% are Brown and 20% are Green. The rest are yellow. How many yellow leaves does he collect?",
        ],
        original_codes=[
            None,
            "def solve():\n    ticket = 8.0\n    popcorn = ticket - 3.0\n    drink = popcorn + 1.0\n    candy = drink / 2.0\n    normal_total = ticket + popcorn + drink + candy\n    deal_price = 20.0\n    answer = normal_total - deal_price\n    return answer",
            "def solve():\n    thursday = 12\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow",
        ],
        answer_values_original=[11.0, 2.0, 15.0],
        composed_question="Let X_i be the answer to Q_i:\nSolve Q_i and use the value of X_i to solve Q_{i+1}. Explain your answer step by step.\nQ1: Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?\nQ2: A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs X_1 dollars, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?\nQ3: Bronson decides to collect Oak Leaves from around his neighborhood. He collects X_2 leaves on Thursday and 13 on Friday. 20% are brown and 20% are green. The rest are yellow. How many yellow leaves does he collect?",
        composed_answer_value=15.300000000000002,
        composed_code="def solve_revised():\n    thursday = 12.5\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow",
        intermediate_answers=[11.0, 12.5, 15.300000000000002],
        rewritten_subquestions=[
            "A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs X_1 dollars, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?",
            "Bronson decides to collect Oak Leaves from around his neighborhood. He collects X_2 leaves on Thursday and 13 on Friday. 20% are brown and 20% are green. The rest are yellow. How many yellow leaves does he collect?",
        ],
        revised_codes=[
            "def solve_revised():\n    ticket = 11.0\n    popcorn = ticket - 3.0\n    drink = popcorn + 1.0\n    candy = drink / 2.0\n    normal_total = ticket + popcorn + drink + candy\n    deal_price = 20.0\n    answer = normal_total - deal_price\n    return answer",
            "def solve_revised():\n    thursday = 12.5\n    friday = 13\n    total = thursday + friday\n    percent_brown = 0.2\n    percent_green = 0.2\n    percent_yellow = 1 - percent_brown - percent_green\n    yellow = total * percent_yellow\n    return yellow",
        ],
    )
]

In [93]:
check = new_res[0]

In [94]:
print(check.composed_question)

Let X_i be the answer to Q_i:
Solve Q_i and use the value of X_i to solve Q_{i+1}. Explain your answer step by step.
Q1: Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?
Q2: A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs X_1 dollars, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?
Q3: Bronson decides to collect Oak Leaves from around his neighborhood. He collects X_2 leaves on Thursday and 13 on Friday. 20% are brown and 20% are green. The rest are yellow. How many yell

In [95]:
print(check.intermediate_answers)

[11.0, 12.5, 15.300000000000002]


In [96]:
check.__dict__.keys()

dict_keys(['ids', 'questions', 'original_codes', 'answer_values_original', 'composed_question', 'composed_answer_value', 'composed_code', 'intermediate_answers', 'rewritten_subquestions', 'revised_codes'])

In [ ]:
for c in check.revised_codes:
    print(c, end="\n---\n")

def solve_revised():
    ticket = 11.0
    popcorn = ticket - 3.0
    drink = popcorn + 1.0
    candy = drink / 2.0
    normal_total = ticket + popcorn + drink + candy
    deal_price = 20.0
    answer = normal_total - deal_price
    return answer
---
def solve_revised():
    thursday = 12.5
    friday = 13
    total = thursday + friday
    percent_brown = 0.2
    percent_green = 0.2
    percent_yellow = 1 - percent_brown - percent_green
    yellow = total * percent_yellow
    return yellow
---


In [ ]:
def solve_revised():
    ticket = 11.0
    popcorn = ticket - 3.0
    drink = popcorn + 1.0
    candy = drink / 2.0
    normal_total = ticket + popcorn + drink + candy
    deal_price = 20.0
    answer = normal_total - deal_price
    return answer


solve_revised()

12.5

In [ ]:
def solve_revised():
    thursday = 12.5
    friday = 13
    total = thursday + friday
    percent_brown = 0.2
    percent_green = 0.2
    percent_yellow = 1 - percent_brown - percent_green
    yellow = total * percent_yellow
    return yellow


solve_revised()

15.300000000000002

## CHAIN 10


In [101]:
from pathlib import Path

In [ ]:
# tgt_path = Path("/Users/matthewho/Documents/research/collabmem/outputs/multihop_compose_test_10.json")
tgt_path = Path(
    "/Users/matthewho/Documents/research/collabmem/outputs/multihop_compose_test_20.json"
)
with open(tgt_path, "r") as f:
    data = json.load(f)

In [116]:
data.keys()

dict_keys(['chains'])

In [117]:
data["chains"][0].keys()

dict_keys(['ids', 'questions', 'original_codes', 'answer_values_original', 'composed_question', 'composed_answer_value', 'composed_code', 'intermediate_answers', 'rewritten_subquestions', 'revised_codes'])

In [118]:
chain10 = MultiHopComposedExample(**data["chains"][0])

In [119]:
print(chain10.composed_question)

Let X_i be the answer to Q_i:
Solve Q_i and use the value of X_i to solve Q_{i+1}. Explain your answer step by step.
Q1: Lily has 5 lottery tickets to sell.  She sells the first ticket for $1.  She then sells each successive ticket for a dollar more than the previous ticket. She plans to keep a $4 profit and give the remaining money as the prize. How much money will the winner of the lottery receive?
Q2: A movie theater company is advertising a deal of a movie ticket, a bucket of popcorn, a drink, and a box of candy for $20. Normally, a movie ticket costs X_1 dollars, a bucket of popcorn costs three dollars less, a drink costs a dollar more than popcorn, and a candy costs half as much as a drink. How many dollars does the deal save a customer who buys a ticket, popcorn, drink, and candy normally?
Q3: Bronson decides to collect Oak Leaves from around his neighborhood. He collects X_2 leaves on Thursday and 13 on Friday. 20% are brown and 20% are green. The rest are yellow. How many yell

In [123]:
chain10.ids[18]

7088

In [126]:
print(problem_dict[chain10.ids[17]])

QAExample(id=1164, question='Hayden has a tank with a small hole in the bottom. The tank starts with 40 gallons of water. It loses 2 gallons of water per hour. Hayden does not add any water for the first two hours. He adds 1 gallon of water to the tank in hour three. He adds three gallons of water to the tank in the fourth hour. How much water is left in the tank at the end of the fourth hour?', answer_text='Over the four hours, the tank loses 2 * 4 = <<2*4=8>>8 gallons of water.\nHayden adds a total of 1 + 3 = <<1+3=4>>4 gallons of water.\nAt the end of the four hours, there is 40 - 8 + 4 = <<40-8+4=36>>36 gallons of water left in the tank.\n#### 36', answer_value=36.0)


In [121]:
# print(chain10.intermediate_answers)
for i, ans in enumerate(chain10.intermediate_answers):
    print(f"Intermediate answer {i+1}: {ans}")

Intermediate answer 1: 11.0
Intermediate answer 2: 12.5
Intermediate answer 3: 15.300000000000002
Intermediate answer 4: 16.830000000000005
Intermediate answer 5: 149.83000000000004
Intermediate answer 6: 899.1500000000001
Intermediate answer 7: 912.1500000000001
Intermediate answer 8: 16418.7
Intermediate answer 9: 8149.35
Intermediate answer 10: 8149.35
Intermediate answer 11: 8159.35
Intermediate answer 12: 89752.85
Intermediate answer 13: 89752.85000000003
Intermediate answer 14: 67314.63750000003
Intermediate answer 15: 15078478.800000006
Intermediate answer 16: 9.947953105189891e-06
Intermediate answer 17: 400.00001989590623
Intermediate answer 18: 396.00001989590623
Intermediate answer 19: 200.00000994795312
Intermediate answer 20: 1009.0000497397656
